In [ ]:
!pip install --user xlsxwriter
!pip install --user ace_tools
!python -m spacy download nl_core_news_sm
!pip install qgrid
!pip Install ipywidgets
!pip install spacy
!python -m spacy download nl_core_news_sm




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 81.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('nl_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 889.2/889.2 kB 32.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.2 MB/s eta 0:00:00
  Created wheel for qgrid: filename=qgrid-1.3.1-py2.py3-none-any.whl size=1761252 sha256=98fb45815a8d16470a0c310f3209ed1c76b598df21ce116a0e7e900b972a5cf7
  Stored in directory: /root/.cache/pip/wheels/0a/83/df/0bccd6bcfb1923fcca59ca6f7b04f6b8d19300faf1a4edcc02
Successfully built qgrid
ERROR: unknown co

# 1. File Preparation and Directory Setup
Ensure all required files are in place:
nl-NL_4_step.xlsx and nl-NL_6_step.xlsx: These are the Excel files containing the original data for the 4-step and 6-step recipes, respectively. They should be located in /content/drive/Shareddrives/BENELUXFR RECIPE CARDS HF 2024/2024/Flemishmaker/indesign-outputs.
Flemishreplacementsheet1.xlsx: This file should contain columns old_word and new_word for words that need to be replaced. Place it in /content/drive/Shareddrives/BENELUXFR RECIPE CARDS HF 2024/2024/Flemishmaker/.
# 2. Define the Replacement Rules
Ensure Flemishreplacementsheet1.xlsx has two columns:
old_word: The word or phrase you want to replace.
new_word: The replacement word or phrase.
Custom replacements are hardcoded in the script, so no further configuration is needed for those.
# 3. Code Execution and Environment
Python and Libraries:
Make sure your environment supports Python and the following libraries:
bash
Copy code
pip install pandas openpyxl
pandas is used for DataFrame manipulation, and openpyxl is needed to handle Excel files.
# 4. Code Structure and Functions
File Existence Check: The code verifies the existence of all required files. If any file is missing, it raises an error.
Replacement Function:
replace_words_in_df: This function performs case-insensitive replacements based on the dictionary created from Flemishreplacementsheet1.xlsx.
custom_replacement: Handles hardcoded custom replacements such as translating specific terms.
replace_pot_with_pan: This new function specifically replaces "Haal uit de pot" with "Haal uit de pan".
# 5. Column Filtering and Sorting
Columns to Keep: Only certain columns are kept in the final DataFrame, as specified in columns_to_keep.
Sorting: After replacements, the DataFrames are sorted by RecipeNumber to maintain correct order.
# 6. Merge and Save
After applying replacements and sorting, the 4-step and 6-step data are merged and saved as a new Excel file named nl-BE_4+6_step.xlsx.
# 7. Running the Code
Execute the code after setting up everything, ensuring you have the correct directory structure and file paths as required.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Flemish maker

In [ ]:
import os
import pandas as pd
import re

# 23-04-2025

# Constants for file paths
BASE_DIR = "/content/drive/Shareddrives/BENELUXFR 2025 Editing Essentials/Flemishmaker/indesign-outputs"
FILE_PATH_4_STEP = os.path.join(BASE_DIR, 'nl-NL_4_step.xlsx')
FILE_PATH_6_STEP = os.path.join(BASE_DIR, 'nl-NL_6_step.xlsx')
REPLACEMENTS_EXCEL_PATH = "/content/drive/Shareddrives/BENELUXFR 2025 Editing Essentials/Flemishmaker/Flemishreplacementsheet1.xlsx"
OUTPUT_FILE_PATH_MERGED = os.path.join(BASE_DIR, '../nl-BE_4+6_step.xlsx')

# Columns to keep in the DataFrame
columns_to_keep = [
    'RecipeNumber', 'RecipeCode', 'RecipeName', 'Recipe_Untertitel', 'TitelStep1',
    'Step1', 'TitelStep2', 'Step2', 'TitelStep3', 'Step3', 'TitelStep4',
    'Step4', 'TitelStep5', 'Step5', 'TitelStep6', 'Step6','IngredientList'
]

# Check if files exist before proceeding
if not os.path.exists(FILE_PATH_4_STEP):
    raise FileNotFoundError(f"The file {FILE_PATH_4_STEP} does not exist.")
if not os.path.exists(FILE_PATH_6_STEP):
    raise FileNotFoundError(f"The file {FILE_PATH_6_STEP} does not exist.")
if not os.path.exists(REPLACEMENTS_EXCEL_PATH):
    raise FileNotFoundError(f"The file {REPLACEMENTS_EXCEL_PATH} does not exist.")

# Function to perform case-insensitive replacement in each DataFrame column
def replace_words_in_df(df, replacements):
    def replace_function(text):
        if isinstance(text, str):
            for old_word, new_word in replacements.items():
                regex = re.compile(r'\b' + re.escape(old_word) + r'\b', re.IGNORECASE)
                text = regex.sub(lambda m: new_word if m.start() == 0 else new_word.lower(), text)
        return text

    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].map(replace_function)
    return df

# Updated function to apply replacements and fix capitalization
def replace_text(text, replacements):
    if text is not None and isinstance(text, str):
        for old, new in replacements.items():
            text = re.sub(fr'\b{re.escape(old)}\b', new, text, flags=re.IGNORECASE)
        text = re.sub(r'([.!?]\s*)([a-z])', lambda m: m.group(1) + m.group(2).upper(), text)
    return text

# Function to apply text replacements for each string column in a DataFrame
def apply_replacements(df, replacements):
    for column in df.columns:
        if df[column].dtype == object:
            df[column] = df[column].map(lambda x: replace_text(x, replacements) if isinstance(x, str) else x)
    return df

# Function to capitalize the first letter of each cell, excluding specific columns
def capitalize_cells(df, exclude_columns=None):
    exclude_columns = exclude_columns or []
    def capitalize_text(text):
        if isinstance(text, str) and text:
            return text[0].upper() + text[1:]
        return text

    for col in df.columns:
        if col not in exclude_columns and df[col].dtype == object:
            df[col] = df[col].map(capitalize_text)
    return df

# Function to replace 'ketjap' with 'ketjap of zoete sojasaus' in steps if not in IngredientList
def conditional_ketjap_replacement(df):
    step_columns = [f"TitelStep{i}" for i in range(1, 7)] + [f"Step{i}" for i in range(1, 7)]
    for index, row in df.iterrows():
        ingredient_list = str(row.get("IngredientList", "")).lower()
        contains_ketjap = "ketjap" in ingredient_list
        if not contains_ketjap:
            for col in step_columns:
                if col in df.columns and isinstance(row[col], str):
                    df.at[index, col] = re.sub(r'\bketjap\b', 'ketjap of zoete sojasaus', row[col], flags=re.IGNORECASE)
    return df

# Read the replacements from the Excel file
replacements_df = pd.read_excel(REPLACEMENTS_EXCEL_PATH)
replacements = dict(zip(replacements_df['old_word'], replacements_df['new_word']))

# Read dataframes with only columns to keep
df_4_step = pd.read_excel(FILE_PATH_4_STEP, usecols=columns_to_keep)
df_6_step = pd.read_excel(FILE_PATH_6_STEP, usecols=columns_to_keep)

# Keep a full copy to retrieve IngredientList
df_4_step_full = pd.read_excel(FILE_PATH_4_STEP)
df_6_step_full = pd.read_excel(FILE_PATH_6_STEP)

# Replace general terms
df_4_step_replaced = replace_words_in_df(df_4_step, replacements)
df_6_step_replaced = replace_words_in_df(df_6_step, replacements)

# Apply manual overrides
custom_replacements = {
    'hapjespan': 'diepe pan',
    'koekenpan': 'pan',
    'haal uit de pot': 'haal uit de pan',
    "haal de omelet uit de pot": "haal de omelet uit de pan",
    "haal daarna uit de pot": "haal daarna uit de pan",
    "haal de pot van het vuur": "haal de pan van het vuur",
    "haal de schnitzel uit de pot": "haal de schnitzel uit de pan",
    "haal de burger uit de pot": "haal de burger uit de pan",
    "haal de garnalen uit de pot": "haal de garnalen uit de pan",
    "haal de pompoenpitten uit de pot": "haal de pompoenpitten uit de pan",
    "haal de amandelen uit de pot": "haal de amandelen uit de pan",
    "haal de pijnboompitten uit de pot": "haal de pijnboompitten uit de pan",
    "kook ruim water": "kook water",
    "breng ruim water": "breng water",
    "breng ondertussen ruim water": "breng ondertussen water",
    "breng op smaak met ruim peper en zout": "breng op smaak met flink wat peper en zout",
    "breng ruim op smaak met zout": "breng flink op smaak met zout",
    "de risotto is gaar zodra de korrel vanbuiten zacht is en nog een lichte bite heeft vanbinnen":
        "de risotto is gaar zodra de korrels vanbuiten zacht zijn, maar nog een hardere kern hebben vanbinnen",
    "bak in dezelfde pot": "bak in dezelfde pan",
    'zodra de pot goed heet is': 'zodra de pan goed is',
    "het bakvet in de pot": "het bakvet in de pan"
}
df_4_step_replaced = apply_replacements(df_4_step_replaced, custom_replacements)
df_6_step_replaced = apply_replacements(df_6_step_replaced, custom_replacements)

# Add IngredientList to DataFrame for conditional ketjap check
df_4_step_replaced["IngredientList"] = df_4_step_full["IngredientList"]
df_6_step_replaced["IngredientList"] = df_6_step_full["IngredientList"]

# Apply ketjap → zoete sojasaus logic if not in ingredients
df_4_step_replaced = conditional_ketjap_replacement(df_4_step_replaced)
df_6_step_replaced = conditional_ketjap_replacement(df_6_step_replaced)

# Drop IngredientList again if not needed in output
df_4_step_replaced = df_4_step_replaced.drop(columns=["IngredientList"])
df_6_step_replaced = df_6_step_replaced.drop(columns=["IngredientList"])

# Merge and finalize
merged_df = pd.concat([df_4_step_replaced, df_6_step_replaced], ignore_index=True)
merged_df = capitalize_cells(merged_df, exclude_columns=['Recipe_Untertitel'])

# Write final output
merged_df.to_excel(OUTPUT_FILE_PATH_MERGED, index=False)
print(f"Merged file has been saved in {OUTPUT_FILE_PATH_MERGED}")


Merged file has been saved in /content/drive/Shareddrives/BENELUXFR 2025 Editing Essentials/Flemishmaker/indesign-outputs/../nl-BE_4+6_step.xlsx


# **Pot and Pan Finder**

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

def find_cells_with_words_in_columns_organized(file_path, columns, word1, word2):
    df = pd.read_excel(file_path)
    missing_columns = [col for col in columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Columns not found in the file: {', '.join(missing_columns)}")
    if 'RecipeNumber' not in df.columns:
        raise ValueError("The file does not contain a 'RecipeNumber' column.")

    matches = []
    for column in columns:
        for idx, cell in df[column].dropna().items():
            if word1 in str(cell) and word2 in str(cell):
                recipe_number = df.loc[idx, 'RecipeNumber']
                matches.append({"RecipeNumber": recipe_number, "Column": column, "Content": cell})

    return pd.DataFrame(matches), df

def display_and_edit_dataframe(result_df):
    """ Create text widgets for each row in the result dataframe for editing. """
    editor_widgets = []
    for index, row in result_df.iterrows():
        text = widgets.Textarea(
            value=row['Content'],
            description=f'{row["RecipeNumber"]} {row["Column"]}:',
            continuous_update=False,
            layout=widgets.Layout(width='100%', height='100px')  # Set height to 100px or more as needed
        )
        editor_widgets.append((index, text))
        display(text)
    return editor_widgets

def save_changes_to_excel(widgets_list, result_df, original_df, file_path):
    """ Update and save the changes to the original DataFrame based on the edited widgets. """
    for index, widget in widgets_list:
        recipe_number = result_df.at[index, 'RecipeNumber']
        column_name = result_df.at[index, 'Column']
        condition = (original_df['RecipeNumber'] == recipe_number)
        correct_index = original_df[condition].index
        if len(correct_index) == 1:
            original_df.at[correct_index[0], column_name] = widget.value
        else:
            raise ValueError(f"Unable to uniquely identify the row to update: RecipeNumber {recipe_number}")
    original_df.to_excel(file_path, index=False)
    print("Changes saved successfully.")

# Load and search data
file_path = "/content/drive/Shareddrives/BENELUXFR 2025 Editing Essentials/Flemishmaker/nl-BE_4+6_step.xlsx"
columns_to_search = ['Step1', 'TitelStep2', 'Step2', 'TitelStep3', 'Step3', 'TitelStep4', 'Step4', 'TitelStep5', 'Step5', 'TitelStep6', 'Step6']
word1 = "pot"
word2 = "pan"

result_df, original_df = find_cells_with_words_in_columns_organized(file_path, columns_to_search, word1, word2)

# Create and display widgets for editing
editor_widgets = display_and_edit_dataframe(result_df)

# Button to save changes
save_button = widgets.Button(description="Save Changes")
save_button.on_click(lambda b: save_changes_to_excel(editor_widgets, result_df, original_df, file_path))
display(save_button)


Textarea(value='Breng water met een snuif zout aan de kook in een pot voor de\xa0pasta.\nVerhit een scheutje o…

Textarea(value='Breng water aan de kook in een pot en kook de rijst 12 - 15 minuten.\xa0\nBreng ondertussen wa…

Textarea(value='Verwarm de oven voor op 200 graden. \nSnijd de ui in halve ringen. \nVerhit een klontje boter …

Textarea(value='Snipper de ui en pers de knoflook of snijd fijn. Snijd de courgette en de paprika in blokjes.\…

Textarea(value='Kook de rijst 12 - 15 minuten in de pot. Giet af en laat uitstomen.\nHoud 1/2 el mayonaise per…

...

Textarea(value='Verhit een pan zonder olie op hoog vuur. Rooster de amandelen tot goudbruin, haal uit de pan e…

Textarea(value='Verhit nog een klontje boter in een pan op middelhoog vuur.\nBak, zodra de boter goed heet is,…

Textarea(value='Verhit een klein klontje boter in een steelpan op middelhoog vuur.\nVoeg de room toe en breng …

Textarea(value='Breng een laagje water aan de kook in een steelpan. Plaats een hittebestendige kom op de pot, …

Button(description='Save Changes', style=ButtonStyle())

# Transformer for upload


In [ ]:
import pandas as pd

# Define the file path to your Excel file
file_path = "/content/drive/Shareddrives/BENELUXFR 2025 Editing Essentials/Flemishmaker/nl-BE_4+6_step.xlsx"

# Load the Excel file
df = pd.read_excel(file_path)

# Create UniqRecipeCode by taking RecipeCode from the dataframe as a string
df["UniqRecipeCode"] = df["RecipeCode"].astype(str)

# Reset the index to preserve a reference to the original row for later title lookup
df_reset = df.reset_index()

# Identify step description columns (exclude title columns)
step_cols = [col for col in df_reset.columns if "Step" in col and "TitelStep" not in col]

# Transform the data to long format, keeping the original index as an identifier
melted_df = df_reset.melt(
    id_vars=["index", "UniqRecipeCode"],
    value_vars=step_cols,
    var_name="StepIndex",
    value_name="Description"
)

# Extract numeric step index from column names (e.g., from "Step1" get 1)
melted_df["StepIndex"] = melted_df["StepIndex"].str.extract("(\d+)").astype(int)

# Filter out rows where Description is missing
melted_df = melted_df[melted_df["Description"].notna()]

# Remove duplicates based on UniqRecipeCode, StepIndex, and Description
melted_df = melted_df.drop_duplicates(subset=["UniqRecipeCode", "StepIndex", "Description"])

# Function to get the title from the original row based on the step index
def get_title(row):
    title_col = f"TitelStep{row['StepIndex']}"
    if title_col in df_reset.columns:
        return df_reset.loc[row["index"], title_col]
    return None

# Apply the function row-wise to assign titles
melted_df["Title"] = melted_df.apply(get_title, axis=1)

# ✅ Filter: Keep only rows where UniqRecipeCode starts with R, Q, or P
melted_df = melted_df[melted_df["UniqRecipeCode"].str.startswith(("R", "Q", "P"))]

# Add locale column for translations (nl-BE)
melted_df["Locale"] = "nl-BE"

# ✅ Add Tip column (initially empty)
melted_df["Tip"] = ""

# Reorder columns and sort the DataFrame
final_df = melted_df[["UniqRecipeCode", "StepIndex", "Locale", "Title", "Description", "Tip"]]
final_df = final_df.sort_values(by=["UniqRecipeCode", "StepIndex"])

# ✅ Remove UniqRecipeCodes that have only one step
step_counts = final_df["UniqRecipeCode"].value_counts()
valid_codes = step_counts[step_counts > 1].index
final_df = final_df[final_df["UniqRecipeCode"].isin(valid_codes)]

# ✅ Function: Format lines into HTML with specific rules for tips and line breaks
def bullet_lines_to_html(text):
    """
    Transforms a multi-line string into HTML format.

    - Unformatted lines become <li> tags within a <ul> list.
    - Lines starting with "Tip:", "Weetje:", or "Gezondheidstip:" are formatted
      with <strong> for the prefix and <em> for the rest of the text.
    - Removes empty lines between bullet points and tip paragraphs.
    """
    if not isinstance(text, str) or not text.strip():
        return text

    lines = text.split('\n')
    html_parts = []
    in_list = False

    for line in lines:
        stripped_line = line.strip()
        is_special_prefix = any(stripped_line.startswith(prefix) for prefix in ("Tip:", "Weetje:", "Gezondheidstip:"))

        if is_special_prefix:
            if in_list:
                html_parts.append("</ul>")
                in_list = False

            # Find the first space after the prefix
            first_space = stripped_line.find(":") + 1
            prefix_text = stripped_line[:first_space]
            italic_text = stripped_line[first_space:].strip()

            # The key change: no <p> or <br /> to remove the blank line space.
            html_parts.append(f"<strong> {prefix_text}</strong> <em>{italic_text}</em>")

        elif stripped_line:
            if not in_list:
                html_parts.append("<ul>")
                in_list = True

            clean_line = stripped_line.lstrip("• ").strip()
            html_parts.append(f"<li>{clean_line}</li>")

        # Blank lines are now just ignored, so no extra HTML is generated for them

    if in_list:
        html_parts.append("</ul>")

    return "\n".join(html_parts)


# 🔧 Apply HTML formatting to the specified column
final_df["Description"] = final_df["Description"].apply(bullet_lines_to_html)

# ✅ Save the transformed data to a new Excel file
output_path = "/content/drive/Shareddrives/BENELUXFR 2025 Editing Essentials/Flemishmaker/nl-BE_4+6_step_transformed.xlsx"
final_df.to_excel(output_path, index=False, sheet_name="StepTranslations")

# ✅ Print clickable link for Colab / Jupyter or readable path elsewhere
print(f"[Transformed data saved to Excel file]({output_path})")


<>:27: SyntaxWarning: invalid escape sequence '\d'
<>:27: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-296414345.py:27: SyntaxWarning: invalid escape sequence '\d'
  melted_df["StepIndex"] = melted_df["StepIndex"].str.extract("(\d+)").astype(int)


[Transformed data saved to Excel file](/content/drive/Shareddrives/BENELUXFR 2025 Editing Essentials/Flemishmaker/nl-BE_4+6_step_transformed.xlsx)
